In [ ]:
# Imports
import pandas as pd
import numpy as np
import requests
import json
from matplotlib import pyplot as plt

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
import pymysql

# Load Train Locations

In [ ]:
'http://api.bart.gov/api/etd.aspx?cmd=etd&orig=BALB&key=MW9S-E7SL-26DU-VV8V#'

In [ ]:
# etd
rest_url = "http://api.bart.gov/api/etd.aspx?cmd=etd&orig=mont&key=MW9S-E7SL-26DU-VV8V&json=y"
response = requests.post(rest_url)
response_json = json.loads(response.content)
raw_etd = pd.io.json.json_normalize(response_json['root']['station'][0]['etd'])


etd = pd.DataFrame()

for i,r in raw_etd.iterrows():
    dest_etd = pd.concat([pd.io.json.json_normalize(est) for est in r['estimate']])
    for c in r.index:
        if c!='estimate':
            dest_etd[c] = r[c]
    etd = pd.concat([etd,dest_etd])
etd

# Connect to RDS

In [ ]:
credentials = {}
with open('credentials.json') as json_data:
    credentials = json.load(json_data)

connection_url = sqlalchemy.engine.url.URL(
    'mysql+pymysql',
    username=credentials['username'],
    password=credentials['password'],
    host=credentials['host'],
    port=credentials['port'],
    database=credentials['database'])
print(connection_url)
engine=create_engine(connection_url)
engine.connect()

In [ ]:
result = engine.execute('select count(*) from fireplacedb.train_observations limit 10;')
result

In [ ]:
for row in result:
    print(row)